LDAM-DRW

In [1]:
! python cifar_train_lorot-E_moe.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LDAM --train_rule DRW --epochs 1 --workers 2

creating folder log/10/cifar10_Moe1_LDAM_DRW_exp_0.01_0
creating folder checkpoint/cifar10_Moe1_LDAM_DRW_exp_0.01_0
cifar_train_lorot-E_moe.py:91: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn('You have chosen a specific GPU. This will completely '
Use GPU: 0 for training
=> creating model 'Moe1'
Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 2997, 1796, 1077, 645, 387, 232, 139, 83, 50]
/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/losses.py:49: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorCompare.cpp:402.)
  output = torch.where(index, x_m, x)
cifar_train_lorot-E_moe.py:340: UserWarning: Implicit dimension choice for softma

In [3]:
! python cifar_train_lorot-E_moe_flip.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LDAM --train_rule DRW --epochs 1 --workers 2

cifar_train_lorot-E_moe_flip.py:91: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn('You have chosen a specific GPU. This will completely '
Use GPU: 0 for training
=> creating model 'Moe1flip'
Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 2997, 1796, 1077, 645, 387, 232, 139, 83, 50]
/home/aristo/miniconda3/envs/pytorch-test/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 44 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/losses.py:49: UserWa

In [4]:
! python cifar_train_lorot-E_moe_sc.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LDAM --train_rule DRW --epochs 1 --workers 2

creating folder log/10/cifar10_Moe1sc_LDAM_DRW_exp_0.01_0
creating folder checkpoint/cifar10_Moe1sc_LDAM_DRW_exp_0.01_0
cifar_train_lorot-E_moe_sc.py:91: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn('You have chosen a specific GPU. This will completely '
Use GPU: 0 for training
=> creating model 'Moe1sc'
Files already downloaded and verified
Files already downloaded and verified
cls num list:
[5000, 2997, 1796, 1077, 645, 387, 232, 139, 83, 50]
/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/losses.py:49: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorCompare.cpp:402.)
  output = torch.where(index, x_m, x)
cifar_train_lorot-E_moe_sc.py:340: UserWarning: Implicit dimension choic

In [1]:
import torchvision
import torch.nn as nn
import torch
import models
from torchsummary import summary

In [2]:
models.__dict__

{'__name__': 'models',
 '__doc__': None,
 '__package__': 'models',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7fa083b4fbb0>,
 '__spec__': ModuleSpec(name='models', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fa083b4fbb0>, origin='/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/models/__init__.py', submodule_search_locations=['/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/models']),
 '__path__': ['/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/models'],
 '__file__': '/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/models/__init__.py',
 '__cached__': '/media/aristo/Data A/documents/kuliah/Project/Localizable-Rotation/Imbalanced/models/__pycache__/__init__.cpython-38.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsi

In [4]:
model = models.BarlowTwins(lorot= True)

In [5]:
print(model)

BarlowTwins(
  (backbone): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      